In [4]:
import requests
import json
import pandas as pd
import os
import re
import sys

from datetime import datetime
from bs4 import BeautifulSoup

sys.path.append("../")
from src.utilities.config_ import scrape_data_path, model_path, summarized_data_path, predicted_data_path, combined_data_path
import src.utilities.utils as utils
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from transformers import BartForConditionalGeneration, BartTokenizer

In [5]:
df = utils.load(os.path.join(combined_data_path, "combined_data_stock.feather"))
df

,title,date,url,category,content,summary,label
0,Citron's Andrew Left asks judge to dismiss SEC...,2024-10-04 22:56:20,https://www.investing.com/news/stock-market-ne...,NVDA,(Reuters) - Citron Capital founder and short s...,Citron Capital founder Andrew Left has asked a...,neutral
1,"Factbox-Guns, transgender rights, porn, regula...",2024-10-04 21:02:04,https://www.investing.com/news/stock-market-ne...,NVDA,(Reuters) - The U.S. Supreme Court is set on M...,The U.S. Supreme Court is set on Monday to beg...,neutral
2,Tech sector projected to see fewer positive ea...,2024-10-04 09:19:45,https://www.investing.com/news/stock-market-ne...,NVDA,Investing.com -- Fewer positive earnings surpr...,"Technology, along with healthcare, were the se...",negative
3,AMD stock: Upcoming AI event ‘could be catch-u...,2024-10-03 14:08:26,https://www.investing.com/news/stock-market-ne...,NVDA,Investing.com -- Bank of America (BofA) analys...,Bank of America (BofA) analysts reiterated a B...,neutral
4,OpenAI closes $6.6 billion funding haul with i...,2024-10-03 05:20:47,https://www.investing.com/news/stock-market-ne...,NVDA,"By Krystal Hu (Reuters) -OpenAI, the company b...","OpenAI, the company behind ChatGPT, has raised...",positive
...,...,...,...,...,...,...,...
2364,"Exclusive-EU's Vestager warns about Apple, Met...",2024-03-19 19:07:07,https://www.investing.com/news/stock-market-ne...,META,By Foo Yun Chee BRUSSELS (Reuters) - EU antitr...,EU antitrust chief Margrethe Vestager warns Ap...,negative
2365,Facebook co-founder Saverin's B Capital raises...,2024-03-19 16:37:27,https://www.investing.com/news/stock-market-ne...,META,(Reuters) -Facebook co-founder Eduardo Saverin...,Facebook co-founder Eduardo Saverin's global i...,neutral
2366,"Midday movers: Nvidia, Coinbase fall; Unilever...",2024-03-19 15:22:01,https://www.investing.com/news/stock-market-ne...,META,None,None,neutral
2367,"Bullish investors pile into EM, European stock...",2024-03-19 08:15:40,https://www.investing.com/news/economy/bullish...,META,By Harry Robertson LONDON (Reuters) - Upbeat i...,Global growth expectations among the fund mana...,neutral


In [43]:
# Function to preprocess text
def preprocess_text(text):
    # Remove extra whitespace and newlines
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces/newlines with a single space
    text = text.strip()  # Remove leading and trailing whitespace
    return text

preprocess_text(df.iloc[4].content) == preprocess_text(df.iloc[16].content) 

True

In [42]:
df.iloc[4].content

"By Krystal Hu\n(Reuters) -OpenAI, the company behind ChatGPT, has raised $6.6 billion from investors, which could value the company at $157 billion and cement its position as one of the most valuable private companies in the world.\nThe funding has attracted returning venture capital investors including Thrive Capital and Khosla Ventures, as well as OpenAI's biggest corporate backer Microsoft (NASDAQ:MSFT), and new participation from Nvidia (NASDAQ:NVDA). \nThe closing of the funds coincides with the company's ongoing restructuring efforts and executive changes, including the abrupt departure of its longtime Chief Technology Officer, Mira Murati, last week.\nAltimeter Capital, Fidelity, SoftBank (TYO:9984), and Abu Dhabi's state-backed investment firm MGX also participated in the round. \nOpenAI Chief Financial Officer Sarah Friar told employees on Wednesday that the company will be able to provide liquidity for them through a tender offer to buy back their shares in the company follo

In [8]:
df2 = utils.load(os.path.join(predicted_data_path, "investing_NVDA_predicted_test.feather"))
df2

,title,date,url,category,content,summary,label
0,Where does the AI semiconductor ecosystem go f...,2024-10-06 11:00:00,https://www.investing.com/news/stock-market-ne...,NVDA,Investing.com -- The trajectory of the AI semi...,Barclays: The AI semiconductor ecosystem is in...,neutral
1,Citron's Andrew Left asks judge to dismiss SEC...,2024-10-04 22:56:20,https://www.investing.com/news/stock-market-ne...,NVDA,(Reuters) - Citron Capital founder and short s...,Citron Capital founder and short seller Andrew...,neutral
2,"Factbox-Guns, transgender rights, porn, regula...",2024-10-04 21:02:04,https://www.investing.com/news/stock-market-ne...,NVDA,(Reuters) - The U.S. Supreme Court is set on M...,The U.S. Supreme Court is set on Monday to beg...,neutral
3,Tech sector projected to see fewer positive ea...,2024-10-04 09:19:45,https://www.investing.com/news/stock-market-ne...,NVDA,Investing.com -- Fewer positive earnings surpr...,"Technology, along with healthcare, were the se...",negative
4,AMD stock: Upcoming AI event ‘could be catch-u...,2024-10-03 14:08:26,https://www.investing.com/news/stock-market-ne...,NVDA,Investing.com -- Bank of America (BofA) analys...,Bank of America (BofA) analysts reiterated a B...,neutral
...,...,...,...,...,...,...,...
985,Cisco Systems Reveals Nexus HyperFabric AI Clu...,2024-06-04 15:08:12,https://www.investing.com/news/assorted/cisco-...,NVDA,None,None of these people have ever been to the Whi...,neutral
986,Can Nvidia really be the only AI winner? Mizuh...,2024-06-04 14:35:13,https://www.investing.com/news/stock-market-ne...,NVDA,"Following the latest updates from Computex, Nv...",Nvidia (NASDAQ:NVDA)'s stock surged 5% to clos...,neutral
987,Musk directed Nvidia to prioritize processor s...,2024-06-04 13:39:48,https://www.investing.com/news/stock-market-ne...,NVDA,Elon Musk reportedly directed Nvidia (NASDAQ:N...,Internal Nvidia emails suggest Elon Musk exagg...,neutral
988,TD Cowen reaffirms Buy rating on NVIDIA follow...,2024-06-04 13:34:38,https://www.investing.com/news/company-news/td...,NVDA,"On Tuesday, TD Cowen maintained a positive sta...",TD Cowen maintained a positive stance on NVIDI...,neutral


In [3]:
# Load the tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
# Load the model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")


# Save the model locally if needed
model.save_pretrained("../model/bart-large-cnn")
tokenizer.save_pretrained("../model/bart-large-cnn")

/opt/anaconda3/envs/market-dash/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('../model/bart-large-cnn/tokenizer_config.json',
 '../model/bart-large-cnn/special_tokens_map.json',
 '../model/bart-large-cnn/vocab.json',
 '../model/bart-large-cnn/merges.txt',
 '../model/bart-large-cnn/added_tokens.json')

In [11]:
# Load the tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
# Load the model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

In [6]:
df = pd.read_csv("../data/scrape/investing_test.csv", index_col=0)
df

,title,date,url,category,content
0,Citron's Andrew Left asks judge to dismiss SEC...,2024-10-04 22:56:20,https://www.investing.com/news/stock-market-ne...,NVDA,(Reuters) - Citron Capital founder and short s...
1,"Factbox-Guns, transgender rights, porn, regula...",2024-10-04 21:02:04,https://www.investing.com/news/stock-market-ne...,NVDA,(Reuters) - The U.S. Supreme Court is set on M...
2,Tech sector projected to see fewer positive ea...,2024-10-04 09:19:45,https://www.investing.com/news/stock-market-ne...,NVDA,Investing.com -- Fewer positive earnings surpr...
3,AMD stock: Upcoming AI event ‘could be catch-u...,2024-10-03 14:08:26,https://www.investing.com/news/stock-market-ne...,NVDA,Investing.com -- Bank of America (BofA) analys...
4,OpenAI closes $6.6 billion funding haul with i...,2024-10-03 05:20:47,https://www.investing.com/news/stock-market-ne...,NVDA,"By Krystal Hu\n(Reuters) -OpenAI, the company ..."
5,S&P 500 ends near flat as more jobs data await...,2024-10-02 23:11:50,https://www.investing.com/news/economy-news/fu...,NVDA,By Caroline Valetkevitch\nNEW YORK (Reuters) -...
6,Demand for NVIDIA's Blackwell is 'insane' - Je...,2024-10-02 22:07:02,https://www.investing.com/news/stock-market-ne...,NVDA,Investing.com -- NVIDIA Corporation (NASDAQ:NV...
7,JPMorgan maintains $155 target on NVIDIA share...,2024-10-02 18:17:53,https://www.investing.com/news/company-news/jp...,NVDA,"\nOn Wednesday, JPMorgan expressed continued c..."
8,Nvidia Corp receives Investment Bank Analyst R...,2024-10-02 18:04:01,https://www.investing.com/news/pro/nvidia-corp...,NVDA,NaN
9,OpenAI closes $6.6B funding at valuation of $1...,2024-10-02 16:55:09,https://www.investing.com/news/pro/openai-clos...,NVDA,NaN


In [12]:
def summarize_with_bart(
        tokenizer,
        model,
        text
):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

    # Generate summary
    summary_ids = model.generate(inputs["input_ids"], max_length=100, min_length=25, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

def get_bart():
    # Load the tokenizer
    tokenizer = BartTokenizer.from_pretrained(os.path.join(model_path, "bart-large-cnn"))
    # Load the model
    model = BartForConditionalGeneration.from_pretrained(os.path.join(model_path, "bart-large-cnn"))

    return tokenizer, model

tokenizer, model = get_bart()

# Apply the function to the 'content' column, only if the value is not NaN
# df['summary'] = df['content'].apply(lambda x: summarize_with_bart(tokenizer, model, x) if pd.notna(x) else x)
# df
print(df.iloc[4].content + "\n\n")
print(summarize_with_bart(tokenizer, model, df.iloc[4].content))

By Krystal Hu
(Reuters) -OpenAI, the company behind ChatGPT, has raised $6.6 billion from investors, which could value the company at $157 billion and cement its position as one of the most valuable private companies in the world.
The funding has attracted returning venture capital investors including Thrive Capital and Khosla Ventures, as well as OpenAI's biggest corporate backer Microsoft (NASDAQ:MSFT), and new participation from Nvidia (NASDAQ:NVDA). 
The closing of the funds coincides with the company's ongoing restructuring efforts and executive changes, including the abrupt departure of its longtime Chief Technology Officer, Mira Murati, last week.
Altimeter Capital, Fidelity, SoftBank (TYO:9984), and Abu Dhabi's state-backed investment firm MGX also participated in the round. 
OpenAI Chief Financial Officer Sarah Friar told employees on Wednesday that the company will be able to provide liquidity for them through a tender offer to buy back their shares in the company following t

In [10]:
for summary in df.summary:
    print(summary, "\n")

Citron Capital founder and short seller Andrew Left has asked a judge to dismiss the lawsuit by the U.S. Securities and Exchange Commission. Left's lawyer, James Spertus, in a court filing argued that the SEC's case "fails to state a claim because it neither alleges a cognizable theory of fraud nor alleges sufficient facts to support the theory alleged" 

The U.S. Supreme Court is set on Monday to begin its new nine-month term. Cases include issues on guns, gender-affirming medical care for transgender minors, online pornography and federal regulatory powers. 

Technology, along with healthcare, were the sectors that reported the highest percentage of positive surprises in the second quarter. The market capitalization in many global mega tech groups fell sharply in July following the unveiling of their last quarterly earnings reports. 

Bank of America (BofA) analysts reiterated a Buy rating for AMD (NASDAQ:AMD) stock ahead of the company's “Advancing AI” event scheduled for October 10

In [8]:
utils.load(os.path.join(scrape_data_path, "investing_INTL_test.feather"))

,title,date,url,category
0,AMD stock: Upcoming AI event ‘could be catch-u...,2024-10-03 14:08:26,https://www.investing.com/news/stock-market-ne...,INTC
1,Intel stock: These are the key investor debates,2024-10-03 11:04:07,https://www.investing.com/news/stock-market-ne...,INTC
2,U.S. stocks higher at close of trade; Dow Jone...,2024-10-02 20:35:59,https://www.investing.com/news/stock-market-ne...,INTC
3,U.S. stocks higher at close of trade; Dow Jone...,2024-09-30 20:37:31,https://www.investing.com/news/stock-market-ne...,INTC
4,Intel's SWOT analysis: chip giant's stock face...,2024-09-30 08:23:49,https://www.investing.com/news/company-news/in...,INTC
5,Investing.com's stocks of the week,2024-09-28 10:00:00,https://www.investing.com/news/stock-market-ne...,INTC
6,"Futures muted, consumer spending and PCE ahead...",2024-09-27 11:04:06,https://www.investing.com/news/economy/futures...,INTC
7,Intel rejected Arm's approach to buy product u...,2024-09-27 08:07:57,https://www.investing.com/news/stock-market-ne...,INTC
8,Intel rejects Arm's approach for product divis...,2024-09-27 06:18:09,https://www.investing.com/news/pro/intel-rejec...,INTC
9,Intel and US to finalise $8.5 billion in chips...,2024-09-27 05:06:43,https://www.investing.com/news/stock-market-ne...,INTC


In [5]:
def format_dates(date_input):
    """
    This function takes a date input which could be 'latest', a single date,
    or a date range, and returns a list of dates in the datetime format.
    """
    today_date = datetime.today()

    if date_input.lower() == "latest":
        return [today_date]
    
    date_parts = date_input.split("|")
    
    if len(date_parts) == 1:
        return [datetime.strptime(date_parts[0], '%d-%m-%Y')]
    
    elif len(date_parts) == 2:
        return [
            datetime.strptime(date_parts[0], '%d-%m-%Y'),
            datetime.strptime(date_parts[1], '%d-%m-%Y')
        ]
    
    else:
        raise ValueError("Invalid date input format")

In [6]:
# Examples of using the function/
print(format_dates("latest"))  # Example 1: 'latest'
print(format_dates("16-09-2023"))  # Example 2: '16-09-2023'
print(format_dates("16-09-2023|20-09-2023"))  # Example 3: '16-09-2023|20-09-2023'

[datetime.datetime(2024, 10, 5, 11, 18, 7, 841700)]
[datetime.datetime(2023, 9, 16, 0, 0)]
[datetime.datetime(2023, 9, 16, 0, 0), datetime.datetime(2023, 9, 20, 0, 0)]


In [15]:
df = pd.read_csv("../data/scrape/investing_NVIDIA_100.csv")
df['date'] = pd.to_datetime(df['date'])
df

,Unnamed: 0,title,date,url,category
0,0,Apple drops out of talks to join OpenAI invest...,2024-09-28 05:40:29,https://www.investing.com/news/stock-market-ne...,NVDA
1,1,"OpenAI sees $11.6 billion revenue next year, o...",2024-09-28 00:15:33,https://www.investing.com/news/stock-market-ne...,NVDA
2,2,Assets in actively managed ETFs top $1 trillio...,2024-09-27 23:20:47,https://www.investing.com/news/stock-market-ne...,NVDA
3,3,UAE foreign minister says his country is doubl...,2024-09-27 19:05:14,https://www.investing.com/news/world-news/uae-...,NVDA
4,4,Nvidia falls as Beijing discourages local comp...,2024-09-27 19:03:24,https://www.investing.com/news/stock-market-ne...,NVDA
...,...,...,...,...,...
965,965,Nvidia Corp receives Investment Bank Analyst R...,2024-06-03 13:14:02,https://www.investing.com/news/pro/itau-bba-ma...,NVDA
966,966,Goldman Sachs: Hedge and mutual funds increase...,2024-06-03 13:08:02,https://www.investing.com/news/stock-market-ne...,NVDA
967,967,NVIDIA stock retains Conviction Buy rating fro...,2024-06-03 12:00:01,https://www.investing.com/news/company-news/nv...,NVDA
968,968,BofA: Four reasons why Big Tech's reign will end,2024-06-03 11:47:02,https://www.investing.com/news/stock-market-ne...,NVDA


In [23]:
a, b = format_dates("16-09-2023|20-09-2023")
df.iloc[0].date < b

False

In [26]:
datetime.strptime('2024-09-28 05:40:29', '%Y-%m-%d %H:%M:%S') > b


True